In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import plot_histogram
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [ ]:
from qiskit.circuit.library import MCXGate
import numpy as np

qc=QuantumCircuit(11,10)

In [ ]:
# Preparing initial coin state: 1/sqrt(2) (|0>+i|1>)
qc.x(10)
qc.h(10)
qc.s(10)
# Preparing initial position state: 1110000000
qc.x(9)
qc.x(8)
qc.x(7)
qc.barrier()

In [ ]:
######### 1st step ########
qc.h(10) # coin toss
qc.barrier()

# Shift to the left: 1110000000 -> 0110000000
qc.x(10) # Modify the control (coin) state : 0 -> 1
qc.cx(10,9)
qc.x(10) # Restore the control (coin) state : 1 -> 0
qc.barrier()

# Shift to the right: 1110000000 -> 0001000000
qc.cx(10,9)
qc.cx(10,8)
qc.cx(10,7)
qc.cx(10,6)
qc.barrier()

# Error mitigation
qc.reset(9)
qc.reset(5)
qc.reset(4)
qc.reset(3)
qc.reset(2)
qc.reset(1)
qc.reset(0)
qc.barrier()

In [ ]:
######## 2nd step ########
qc.h(10) #coin toss
qc.barrier()

# Shift to the left: 0110000000 -> 1010000000, 0001000000 -> 11100000
qc.x(10) # Flip the control (coin) state
qc.cx(10,9)
qc.cx(10,8)
qc.ccx(10,6,7)
qc.ccx(10,8,6)
qc.x(10) # Restore
qc.barrier()

# Shift to the right: 0110000000 -> 1110000000, 0001000000 -> 1001000000
qc.cx(10,9)
qc.barrier()

# Error mitigation
qc.reset(5)
qc.reset(4)
qc.reset(3)
qc.reset(2)
qc.reset(1)
qc.reset(0)
qc.barrier()

In [ ]:
######## 3rd step ########

qc.h(10) # coin toss
qc.barrier()

# Shift to the left: 1010000000 -> 0010000000, 1110000000 -> 0110000000, 1001000000 -> 0001000000
qc.x(10) # Flip the control qubit
qc.cx(10,9)
qc.x(10) # Restore
qc.barrier()

# Shift to the right: 1010000000 -> 0110000000, 1110000000 -> 0001000000, 1001000000 -> 0101000000
qc.ccx(10,8,6)
qc.ccx(10,8,7)
qc.cx(10,8)
qc.cx(10,9)
qc.barrier()

# Error mitigation
qc.reset(9)
qc.reset(5)
qc.reset(4)
qc.reset(3)
qc.reset(2)
qc.reset(1)
qc.reset(0)
qc.barrier()


In [ ]:
######## 4th step ########
qc.h(10) # coin toss
qc.barrier()

# Shift to the left: 0010000000  0001000000  0110000000  0101000000 into
#                    1100000000  1110000000  1010000000  1001000000
qc.x(10)
qc.cx(10,9)
qc.cx(10,8)
qc.ccx(10,8,7)
qc.append(MCXGate(3),[10,8,7,6])
qc.x(10)
qc.barrier()

# Shift to the right: 0010000000  0001000000  0110000000  0101000000 into
#                     1010000000  1001000000  1110000000  1101000000
qc.cx(10,9)
qc.barrier()

# Error mitigation
qc.reset(5)
qc.reset(4)
qc.reset(3)
qc.reset(2)
qc.reset(1)
qc.reset(0)
qc.barrier()


In [ ]:
######## 5th step ########
qc.h(10) # coin toss
qc.barrier()

# Shift to the left: 1100000000  1010000000  1110000000  1001000000  1101000000 into
#                    0100000000  0010000000  0110000000  0001000000  0101000000
qc.x(10)
qc.cx(10,9)
qc.x(10)
qc.barrier()

# Shift to the right: 1100000000  1010000000  1110000000  1001000000  1101000000 into
#                     0010000000  0110000000  0001000000  0101000000  0011000000
qc.append(MCXGate(3), [10,8,7,6])
qc.ccx(10,8,7)
qc.cx(10,8)
qc.cx(10,9)
qc.barrier()

# Error mitigation
qc.reset(9)
qc.reset(9)
qc.reset(5)
qc.reset(4)
qc.reset(3)
qc.reset(2)
qc.reset(1)
qc.reset(0)
qc.barrier()

# Measurement
qc.measure(range(10),range(10))

# fake backend: FakeCairoV2

#from qiskit_ibm_provider import IBMProvider
#from qiskit.providers.ibmq import IBMQ
#from qiskit_aer import AerSimulator
#from qiskit.providers.fake_provider import FakeCairoV2

#device_backend = FakeCairoV2()
#sim_cairov2 = AerSimulator.from_backend(device_backend)

#job = sim_cairov2.run(transpile(qc,sim_cairov2), shots=5000)
#result = job.result()
#counts = result.get_counts(qc)
#plot_histogram(counts)


In [ ]:
# real backend
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider(instance='q-info-hackathon/team-22/main')
backend = provider.get_backend('ibm_canberra')

options = Options()
options.optimization_level = 1
options.resilience_level = 1

job = backend.run(transpile(qc,backend), shots=5000)
result = job.result()
counts = result.get_counts(qc)
plot_histogram(counts)

In [ ]:
def reverse_binary(bin_string):
    # Reverse the string and return it
#    bin_string = bin_string[:-1]
    return bin_string[::-1]
    
# Dictionary comprehension to create a new dictionary with the reversed keys
reversed_counts = {reverse_binary(k): v for k, v in counts.items()}

###################################

decimal_numbers = {}

for binary, count in reversed_counts.items():
    decimal = int(binary, 2)
    decimal_numbers[decimal] = count
    
print(decimal_numbers)
#plot_histogram(decimal_numbers)
plot_histogram(decimal_numbers, title='step 3 (real)', filename='step3_hist_real')